In [1]:
%run preprocecssing_helpers.ipynb
import os
import time
import mne
from mne.preprocessing import ICA
import numpy
import matplotlib.pyplot as plt
# %matplotlib inline
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
matplotlib.rcParams.update({'font.size': 15})
# matplotlib.rcParams.

In [ ]:
# path = 'F:\StroopEEG'
# path = 'E:\EEGLAB'


# eeglab_files = os.listdir(path)
# eeglab_dict = search_folders(path, eeglab_files)
# eeglab_files = paths_of_eeglab_files(path, eeglab_dict)
# eeglab_files

In [83]:
new_stim_code_event_ids = {
     
    "b-f2-wrd":1,
    "b-f1-wrd":2,
    "b-tg-wrd":3,

    "b-f2-spk":4,
    "b-f1-spk":5,
    "b-tg-spk":6,

    "b-f2-pic":7,
    "b-f1-pic":8,
    "b-tg-pic":9,

    "b-f2-snd":10,
    "b-f1-snd":11,
    "b-tg-snd":12,

    "f-f2-wrd":13,
    "f-f1-wrd":14,
    "f-tg-wrd":15,

    "f-f2-spk":16,
    "f-f1-spk":17,
    "f-tg-spk":18,

    "f-f2-pic":19,
    "f-f1-pic":20,
    "f-tg-pic":21,

    "f-f2-snd":22,
    "f-f1-snd":23,
    "f-tg-snd":24
}

In [93]:
# raw = mne.io.read_raw_eeglab(eeglab_files[38],preload = True)

In [15]:
def list_eeglab_files(root_path):
    """
    NOTE: must perform magic command %run preprocecssing_helpers.ipynb before using this method.
    
    Description: 
        Lists all eeglab .set files within a directory. 
        Assuming that the file format is: 
        Root Folder -> Subject Folder -> eeglab file contents
        example: 
        EEGLAB (root) -> SA20140220 (subject) -> 20140220_1009.set
        or 
        F:\\EEGLAB\\SA20140220\\20140220_1009.set
    
    Variables: 
        root_path: the folder containing all subject folders
    ----
    Returns:
        List of file .set path locations.
    """
    
    eeglab_files_list = os.listdir(root_path)
    eeglab_dict = search_folders(root_path, eeglab_files_list)
    eeglab_files = paths_of_eeglab_files(path, eeglab_dict)
    
    return eeglab_files

In [16]:
# eeglab_files = list_eeglab_files(path)

In [17]:
def find_events(eeglab):
    """
    Description:
        returns the events of an eeglab .set object
        
    Variables:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
    ------
    returns: np array of 3d lists
    """
    return mne.find_events(eeglab, output= 'offset',
#                               consecutive= True,
                              min_duration=1/eeglab.info['sfreq'], 
                              shortest_event= 0, 
                              initial_event=False)
    

In [92]:
# events = find_events(raw)
# events

In [19]:
def get_trials(events):
    """
    Descriptions:
        returns a list of trials where each trial contains timestamp, offset and event_id
    
    Variables:
        events: A np array of 3d lists. use mne.find_events or custom find_events(eeglab) method already written.
    -----
    returns: np array of lists
    """
    trials = []
    current_trial = []
    for event in events:
        if event[-1] == 31 and current_trial != []:
            trials.append(current_trial)
            current_trial = []
            current_trial.append(event)
        else:
            current_trial.append(event)
            
    trials.append(current_trial) # the last trial does not have an indicator, so append it after the loop ends
    return trials

In [67]:
# trials = get_trials(events)

In [91]:
# trials

In [22]:
def filter_bad_trials(eeglab):
    """
    Description:
        Filters out all events that are not within the 1-24 stim-code definitions.
        Also filters for trials that have a reaction time greater than .500ms as the subject 
        failed to click in time and was marked incorrect anyway.
        
    Variables:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
    ------
    returns:
        new_events: 
            np array of 3d list of events 
    """
    
    events = find_events(eeglab)
    trials = get_trials(events)
    button_clicks = [131,132]
    accuracy = [129,130]
    found_first_stim_code = False
    found_third_stim_code = False
    isEnd_of_trial = False
    new_events = []
    good_trials = []
    bad_trials_count = 0
    for trial in trials:
#         print(trial)
        for event in trial:
            # find first stim-code
            if event[-1] <= 24 and found_first_stim_code == False: 
                first_stim_code = event
                found_first_stim_code = True
#                 print("found first stim-code: {0}".format(first_stim_code))

            # find third stim-code
            if found_first_stim_code and event[-1] <= 24:
                #the third stim-code is one less than the first. 
                if event[-1] + 1 == first_stim_code[-1]:
                    third_stim_code = event
                    found_third_stim_code = True
            
            if event[-1] in button_clicks:
                button_click = event        
                
            if event[-1] in accuracy:
                #end of trial
                if event[-1] == 130: # incorrect
                    button_click = event
                    time_diff = (button_click[0] - third_stim_code[0]) * (1/512) #compute time in seconds
                    if time_diff > 1:
                        print("time difference {0}".format(time_diff))
                        print(third_stim_code)
                        print(button_click)
                        bad_trials_count+=1
                    else:
                        #store this trial
                         good_trials.append(trial)
                else:
                    good_trials.append(trial)
                found_first_stim_code = False
                found_third_stim_code = False
                
    print("\n **{0} bad trials dropped**".format(bad_trials_count))
    return np.concatenate([x for x in good_trials])

In [69]:
# filtered_events = filter_bad_trials(raw)

4888 events found
Event IDs: [  3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20
  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139 140 141
 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
time difference 1.134765625
[120626      0     10]
[121207      0    130]
time difference 1.1484375
[260711      0      7]
[261299      0    130]
time difference 3.2265625
[325286      0     16]
[326938      0    130]

 **3 bad trials dropped**


In [70]:
# filtered_events

array([[ 11442,      0,     31],
       [ 11445,      0,    143],
       [ 11448,      0,    163],
       ...,
       [725544,      0,    132],
       [725698,      0,    134],
       [725701,      0,    129]], dtype=int64)

In [24]:
def events_with_stim_codes(events):
    """
    Description:
        Method will return an mne events ndarray containing only 
        stim-codes (event_ids 1-24).
    Variables:
        events: list of 3d lists 
    -----
    Returns:
        new_events list of 3d lists containing events of stim-codes    
    """
    new_events = []
    for event in events:
        if event[-1] <= 24:
            new_events.append(np.array(event))
            
    return np.array(new_events)
    

In [71]:
# stim_code_events = events_with_stim_codes(filtered_events)

In [72]:
# stim_code_events

array([[ 11468,      0,     20],
       [ 11701,      0,     24],
       [ 11994,      0,     19],
       ...,
       [724921,      0,     23],
       [725189,      0,     21],
       [725439,      0,     22]], dtype=int64)

In [25]:
# some eeglab .set files may not have seen ALL of the 1-24 stim-code events.
# write method to return a proper event_id dictionary
def get_stim_code_event_ids(events, event_ids):
    """
    Description:
        return dictionay of event_ids that match the criteria of the eeglab file.
        Some files may not have all stim-code events, and to avoid errors when creating epochs, 
        this method will return a dictionary of compatible event_ids.
        
    Variables:
        events: ndarray of ints (n_events, 3)
        event_ids: dictionary of event_ids 
            ex. {'stim-code': 1}
    -----
    return: dictionary of event_ids
    """
    unique_events = np.unique(events[:,-1])
    event_id_values = event_ids.values()
    event_dict = {}
    
    for event_id in unique_events:
        if event_id in event_id_values:
            key = list(event_ids.keys())[event_id-1]
            event_dict[key] = event_id
            
    return event_dict

In [26]:
def artifact_removal(eeglab):
    """
    Description:
        Perform notch filter, frequenc filter, and ICA 
        
    Variables: 
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
    returns:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object

    """
    
    raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
    raw.filter(5,100, fir_design='firwin')
#     raw.resample(256, n_jobs=10)
    ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
    ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
    ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
                 ecg_score_func='pearsonr', skew_criterion=None, 
                 kurt_criterion=None, var_criterion=None,add_nodes=None)
    
    return ica.apply(raw)

In [77]:
def create_epochs(eeglab, events, event_id):
    """
    Description: 
        create mne.epoch object from eeglab data.
    
    Variables:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
        events: array of int, shape (n_events, 3)
        event_id: dictionary of events 
            Keys are name of event as string, value is event_id as an int.
    
    returns:
        mne.epoch object
            
    """
    try:
        tmax = (1/raw.info['sfreq']) * 256 # tmax 1/sfrq * desired epoch length
        epoch = mne.Epochs(raw, 
                      events=events,
                      event_id = new_stim_code_event_ids, 
                      preload = True, 
                      tmin=0, 
                      tmax=tmax) 
        return epoch
    except:
        print("Error in method create_epochs")

In [81]:
def remove_artifacts_and_create_epochs(eeglab):
    """
    Description:
        Handles the start to finish process of finding events, event_ids, 
        artifact removal, and creating epoch object.
    
    Variables:
        eeglab: mne.io.eeglab.eeglab.RawEEGLAB object
    
    Returns:
        mne.epochs object
    """
    event_create = False
    stim_code_event_create = False
    stim_code_events_id_create = False
    raw_ica_create = Falsehttp://localhost:8888/notebooks/seniorprojecteeg-master-fork/preprocessing/Artifact_Removal/Batch_ArtifactFilter_Epoch.ipynb#
    try:
        events = filter_bad_trials(eeglab)
        event_create = True
        stim_code_events = events_with_stim_codes(events)
        stim_code_event_create = True
        stim_code_event_ids = get_stim_code_event_ids(stim_code_events,new_stim_code_event_ids)
        stim_code_events_id_create = True
        raw_ica = artifact_removal(eeglab= eeglab)
        raw_ica_create = True

        return create_epochs(eeglab= raw_ica, events= stim_code_events, event_id= stim_code_event_ids)
    except:
        print("Error in method remove_artifacts_and_create_epochs")
        print("info:\n event_create: {0} \n stim_code_event_create: {1} \n \
              stim_code_Events_id_Create: {2} \n raw_ica_create: {3} \n".format(event_create,
                                                                               stim_code_event_create,
                                                                               stim_code_events_id_create,
                                                                               raw_ica_create))

In [94]:
# epoch = remove_artifacts_and_create_epochs(eeglab=raw)

In [ ]:
# get_stim_code_event_ids(new_events, new_stim_code_event_ids)

In [ ]:
# list(new_stim_code_event_ids.keys())

In [ ]:
# new_stim_code_event_ids.values()

In [ ]:
# new_events = events_with_stim_codes(filtered_events)
# np.unique(new_events[:,-1])

In [ ]:
# for set_file in eeglab_files[0]:
#     try:
#         start = time.time()
#         raw = mne.io.read_raw_eeglab(set_file, preload=True)
#         raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
#         raw.filter(5,100, fir_design='firwin')
#         raw.resample(256, n_jobs=10)
#         ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
#         ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
#         ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
#                      ecg_score_func='pearsonr', skew_criterion=None, 
#                      kurt_criterion=None, var_criterion=None,add_nodes=None)
#         raw_ica = ica.apply(raw)

#         m_event = mne.find_events(raw_ica, min_duration=1/raw.info['sfreq'], shortest_event= 1, initial_event=True)
#         epoch = mne.Epochs(raw, events=m_event, event_id = event_id, preload = True)
# #         epoch.save(set_file.split('\\')[-1].split('.')[0] + "_epo.fif", verbose = True)
#         end = time.time()
#         print("SUCESSFULLY LOADED time elapsed: {0}", (end - start))
#     except :
#         print("***Error converting file: {0}***".format(set_file))
        

In [ ]:
# start = time.time()
# # raw = mne.io.read_raw_eeglab(, preload=True)
# raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
# raw.filter(5,100, fir_design='firwin')
# raw.resample(256, n_jobs=10)
# ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
# ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
# ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
#              ecg_score_func='pearsonr', skew_criterion=None, 
#              kurt_criterion=None, var_criterion=None,add_nodes=None)
# raw_ica = ica.apply(raw)

# # m_event = mne.find_events(raw_ica, min_duration=1/raw.info['sfreq'], shortest_event= 0.5, initial_event=True)
# epoch = mne.Epochs(raw, events=new_events, event_id =new_stim_code_event_ids , preload = True, tmin=0, tmax=1,)
# end = time.time()
# print("time elapsed: {0}", (end - start))

## check if saving and loading epoch file works.

In [ ]:
# os.chdir(epoch_destination)

In [ ]:

# epoch.save('20131216_1310_epo.fif',verbose = True)

In [ ]:
# epoch_files = os.listdir(epoch_destination)

In [ ]:
# epoch_file = epoch_files[0]
# epoch_ = mne.read_epochs(epoch_file,preload=True)

In [ ]:
# epoch_.plot(title="epochs");

In [ ]:
# for set_file in eeglab_files:
#     print(set_file.split('\\')[-1].split('.')[0] + "_epo.fif")

In [ ]:
# # start = time.time()
# raw = mne.io.read_raw_eeglab(eeglab_files[1], preload=True)
# raw.notch_filter(np.arange(60,241,60), filter_length='auto', phase = 'zero')
# raw.filter(5,100, fir_design='firwin')
# raw.resample(256, n_jobs=10)
# ica = ICA(n_components=None, max_iter = 1000, random_state = 42)
# ica.fit(raw,picks = None, decim=3, reject=dict(mag=4e-12, grad=4000e-13))
# ica.detect_artifacts(raw,start_find=None, stop_find=None, ecg_ch=None, 
#              ecg_score_func='pearsonr', skew_criterion=None, 
#              kurt_criterion=None, var_criterion=None,add_nodes=None)
# raw_ica = ica.apply(raw)

# m_event = mne.find_events(raw_ica, min_duration=1/raw.info['sfreq'], shortest_event= 1.5, initial_event=True)
# epoch = mne.Epochs(raw, events=m_event, event_id = event_id, preload = True)

## Convert Raw EEGLAB Data

In [ ]:
# #get eeglab data
# # path = 'C:\\Users\Emmanuil\Desktop\School\Fall 2018\Senior Project\EEGLAB'
# path = 'D:\\EEGLAB'
# eeglab_files = list_eeglab_files(path)

In [ ]:
# eeglab_files

In [96]:
# # save new epoch files in the directory specified below. 
# os.chdir('E:\correct_time_epochs')
# montage = mne.channels.read_montage(kind ='ANT_DukeWaveGuard_128_electrode_montages_updated_V4')

# for file in eeglab_files[40:50]:
# for file in eeglab_files[50:]:
#     try:
#         read_file = False
#         created_epoch = False
#         set_montage = False
#         saved_epoch = False
#         raw = mne.io.read_raw_eeglab(file, preload = True)
#         read_file = True
#         epoch = remove_artifacts_and_create_epochs(raw)
#         created_epoch = True
#         epoch.set_montage(montage)
#         set_montage = True
#         epoch.save(file.split('\\')[-1].split('.')[0] + "_epo.fif", verbose = True)
#         saved_epoch = True
#     except:
#         print("Could not convert {0}".format(file))
#         print("info:\n read_file: {0} \n created_epoch: {1} \n set_montage: {2} \n saved_epoch: {3} \n".format(read_file,
#                                                                                                                created_epoch,
#                                                                                                                set_montage,
#                                                                                                                saved_epoch))

Reading E:\EEGLAB\SA20140415\20140415_1137.fdt
Reading 0 ... 889656  =      0.000 ...  1737.609 secs...
6344 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
time difference 1.29296875
[238511      0     10]
[239173      0    130]
time difference 1.037109375
[306314      0      1]
[306845      0    130]
time difference 1.064453125
[314635      0      1]
[315180      0    130]
time difference 1.818359375
[458942      0      4]
[459873      0    130]
time difference 1.169921875
[477034      0      4]
[477633      0    130]
time difference 1.935546875
[511855      0     19]
[512846      0    130]
time difference 3.73046875
[519218      0     19]
[521128      0    130]
time difference 1.021484375
[577146      0     19]
[577669      0    130]
time difference 1.474609375
[585864      0     19]
[586619   

<ipython-input-96-d4efa7d53f0d>:15: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: ['Nasium', 'LL4', 'L12', 'VEOG']. Their position has been left untouched.
  epoch.set_montage(montage)


Reading E:\EEGLAB\SA20140415\20140415_1309.fdt
Reading 0 ... 692722  =      0.000 ...  1352.973 secs...
5245 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
time difference 2.001953125
[42976     0    13]
[44001     0   130]
time difference 1.365234375
[48941     0    13]
[49640     0   130]
time difference 1.3671875
[51988     0    13]
[52688     0   130]
time difference 1.078125
[118763      0     16]
[119315      0    130]

 **4 bad trials dropped**
Setting up band-stop filter
Filter length of 3379 samples (6.600 sec) selected
Setting up band-pass filter from 5 - 1e+02 Hz
l_trans_bandwidth chosen to be 2.0 Hz
h_trans_bandwidth chosen to be 25.0 Hz
Filter length of 845 samples (1.650 sec) selected
Fitting ICA to data using 128 channels (please be patient, this may take a while)
Inferring max

C:\Users\Emmanuil\Anaconda3\envs\mne\lib\site-packages\sklearn\decomposition\fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


Fitting ICA took 434.6s.
    Searching for artifacts...
Artifact indices found:
    
Ready.
Transforming to ICA space (128 components)
Zeroing out 0 ICA components
1726 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 1726 events and 257 original time points ...
0 bad epochs dropped
The following EEG sensors did not have a position specified in the selected montage: ['Nasium', 'LL4', 'L12', 'VEOG']. Their position has been left untouched.


...

C:\Users\Emmanuil\Anaconda3\envs\mne\lib\site-packages\sklearn\decomposition\fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


Fitting ICA took 316.9s.
    Searching for artifacts...
Artifact indices found:
    
Ready.
Transforming to ICA space (128 components)
Zeroing out 0 ICA components
Error in method create_epochs
Could not convert E:\EEGLAB\SA20140430\20140430_1309.set
info:
 read_file: True 
 created_epoch: True 
 set_montage: False 
 saved_epoch: False 

Reading E:\EEGLAB\SA20140430\20140430_1501.fdt
Reading 0 ... 1163169  =      0.000 ...  2271.814 secs...
7733 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  31 129 130 131 132 133 134 135 136 137 138 139
 140 141 142 143 144 145 155 156 157 158 159 160 161 162 163 164 165]
time difference 1.09765625
[130435      0     13]
[130997      0    130]
time difference 1.703125
[338418      0     19]
[339290      0    130]
time difference 1.166015625
[422268      0     22]
[422865      0    130]
time difference 1.0703125
[429475      0      7]
[430023      0    130]
time difference 1.25

C:\Users\Emmanuil\Anaconda3\envs\mne\lib\site-packages\sklearn\decomposition\fastica_.py:121: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


Fitting ICA took 637.1s.
    Searching for artifacts...
Artifact indices found:
    
Ready.
Transforming to ICA space (128 components)
Zeroing out 0 ICA components
2521 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 2521 events and 257 original time points ...
0 bad epochs dropped
The following EEG sensors did not have a position specified in the selected montage: ['Nasium', 'LL4', 'L12', 'VEOG']. Their position has been left untouched.


<ipython-input-96-d4efa7d53f0d>:15: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: ['Nasium', 'LL4', 'L12', 'VEOG']. Their position has been left untouched.
  epoch.set_montage(montage)


In [ ]:
# # add montage files to epoch files.
# epoch_path = 'D:\\stim_code_epochs'
# epoch_files = os.listdir(epoch_path)

In [ ]:
# montage = mne.channels.read_montage(kind ='ANT_DukeWaveGuard_128_electrode_montages_updated_V4')

In [ ]:
# os.chdir('D:\stim_code_epochs')
# for epoch_file in epoch_files[1:]:
#     epoch = mne.read_epochs(os.path.join(epoch_path, epoch_file), preload = True)
#     epoch.set_montage(montage=montage)
#     epoch.save(epoch_file.split('\\')[-1].split('.')[0] + "_epo.fif", verbose = True)
    